In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [2]:
import os
import pandas as pd
import numpy as np

def encode_data(fileprefix, data_dir="/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS", 
                features=("1mer",), w=0):
    try:
        # Load 1mer data
        ohe_file = os.path.join(data_dir, "ROHS_MWORDS_1mer", f"{fileprefix}_1mer.tsv")
        ohe_df = pd.read_csv(ohe_file, header=None, sep="\t")
        ohe_data = {row[0]: {'aff': float(row[0].split('__')[1]), 'feat': row.iloc[1:].tolist()} for _, row in ohe_df.iterrows()}
        print(f"1mer data loaded from {ohe_file}... \n")
    except FileNotFoundError as fe:
        raise FileNotFoundError(f"File {ohe_file} not found: {fe}")

    combined_data = ohe_data.copy()

    # Load additional features if specified
    if len(features) > 1:
        for feature in features[1:]:
            print(f"Will load the additional feature {feature}")
            try:
                flex_feat_file = os.path.join(data_dir, "ROHS_MWORDS_FLEX", f"{fileprefix}_{feature}_{w}.tsv")
                flex_df = pd.read_csv(flex_feat_file, sep="\t", header=None)

                for ids in ohe_data.keys():
                    flex_df_row = flex_df[flex_df[0] == ids]
                    if not flex_df_row.empty:
                        flex_df_feat = np.array(flex_df_row.iloc[0, 1:].tolist()) # first column is id

                        # Scale the flex features
                        min_value = np.min(flex_df_feat)
                        std_value = np.std(flex_df_feat)
                        if std_value != 0:
                            scaled_flex_feat = (flex_df_feat - min_value) / std_value
                        else:
                            scaled_flex_feat = flex_df_feat  # If std is 0, scaling is not possible

                        # Calculate interaction terms for this feature using NumPy
                        interaction_terms = scaled_flex_feat[:-1] * scaled_flex_feat[1:]

                        combined_data[ids]['feat'].extend(scaled_flex_feat.tolist() + interaction_terms.tolist())
                print(f"{feature} data loaded from {flex_feat_file}... \n")
            except FileNotFoundError as fe:
                raise FileNotFoundError(f"File {flex_feat_file} not found: {fe}")
            except Exception as e:
                raise Exception(f"Error in extending feature {feature} from {flex_feat_file}: {e}")

    # Define X, y
    X = np.array([combined_data[x]['feat'] for x in combined_data.keys()])
    y = np.array([combined_data[x]['aff'] for x in combined_data.keys()])

    return X, y

# Example usage:
# X, y = encode_data("example_prefix", features=("1mer", "2mer", "3mer"))

In [15]:
def nested_ridge_cv(X, y, fileid, pos, features, outer_cv_folds=10, inner_cv_folds=5):
    
#     alphas = np.linspace()
    alphas = np.logspace(-1, 6, 5000)

    
    outer_cv = KFold(n_splits=outer_cv_folds, shuffle=True, random_state=42)
    outer_results = []

    for train_idx, test_idx in outer_cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        
        inner_cv = KFold(n_splits=inner_cv_folds, shuffle=True, random_state=42)
        ridge_cv_model = make_pipeline(StandardScaler(with_mean=False),
                                       RidgeCV(alphas=alphas, cv=inner_cv, scoring='r2'))
        
        
        ridge_cv_model.fit(X_train, y_train)
        best_alpha = ridge_cv_model.named_steps['ridgecv'].alpha_
        
        y_test_pred = ridge_cv_model.predict(X_test)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        
        
        result = {
            'fileid': fileid,
            'pos': pos,
            'features': features,
            'r2_test': r2_test,
            'mse_test': mse_test,
            'mae_test': mae_test,
            'best_alpha': best_alpha
        }
        outer_results.append(result)

    return outer_results

In [16]:
from itertools import product


ohe = ["1mer"]
# all flexixbility features
# features = ['DNaseI', 'NPP', 'twistDisp', 'stiffness', 'trxDi']
# feature_sets = [('1mer', )] + [x for x in product(ohe, features)]
features = [('1mer', ), ('1mer', "DNaseI", 'twistDisp', 'NPP', 'stiffness', 'trxDi')]
results_df_f_combined = []

features

[('1mer',), ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi')]

In [17]:
META_FILE = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/ROHS_MWORDS.csv"

def read_metadata(file_path):
    metadata = []

    with open(file_path, 'r') as file:
        for line in file:
            family, tf, motif1, motif2, length, score = line.strip().split(',')
            suffix = f"{motif1}_{motif2}_{length}_{score}"

            metadata.append({
                'family': family,
                'tf': tf,
                'suffix': suffix})
            
    return metadata


FILE_IDS = read_metadata(META_FILE)
FILE_IDS

[{'family': 'bHLH', 'tf': 'Bhlhb2', 'suffix': 'TCAAGG20NGAA_CACGTG_14_4'},
 {'family': 'bHLH', 'tf': 'MAX', 'suffix': 'TGACCT20NGA_CACGTG_12_3'},
 {'family': 'bHLH', 'tf': 'TCF3', 'suffix': 'TACCCG20NCCC_CACCTG_12_3'},
 {'family': 'bHLH', 'tf': 'TCF4', 'suffix': 'TGCGAA20NGA_CACCTG_14_3'},
 {'family': 'bHLH', 'tf': 'TFAP4', 'suffix': 'TCACGT20NGCA_CAGCTG_12_4'},
 {'family': 'bHLH', 'tf': 'TFE3', 'suffix': 'TACCGT20NTA_CACGTG_12_3'},
 {'family': 'bHLH', 'tf': 'TFEB', 'suffix': 'TAGTTT20NCG_CACGTG_14_4'},
 {'family': 'bHLH', 'tf': 'USF1', 'suffix': 'TGACGA20NGCA_CACGTG_14_4'},
 {'family': 'bZIP', 'tf': 'ATF4', 'suffix': 'TGGAAT20NGA_TGAYGC_12_3'},
 {'family': 'bZIP', 'tf': 'Cebpb', 'suffix': 'TCCTAA20NATC_TTRCGC_12_3'},
 {'family': 'bZIP', 'tf': 'CEBPB', 'suffix': 'TGGACA20NGA_TTRCGC_12_4'},
 {'family': 'bZIP', 'tf': 'CEBPD', 'suffix': 'TAATGA20NCG_TTRCGC_12_3'},
 {'family': 'bZIP', 'tf': 'CEBPE', 'suffix': 'TGAGGC20NGA_TTRCGC_12_4'},
 {'family': 'bZIP', 'tf': 'CEBPG', 'suffix': 'TAGACG2

In [ ]:
for record in FILE_IDS:
    
    print(f"Running for file: {record} \n")
    for fset in features:

        print(f"current feature is: {fset}...\n")
        file_id = f"{record['family']}_{record['tf']}_{record['suffix']}"
        
        # load data
        X, y = encode_data(fileprefix=file_id, features=fset)
        
        # model
        res = nested_ridge_cv(X = X, y = y, 
                              fileid = file_id, 
                              pos = "all", 
                              features = fset, 
                              outer_cv_folds=10, 
                              inner_cv_folds=5)
        
        print(res)

        results_df_f_combined.append(res)

Running for file: {'family': 'bHLH', 'tf': 'Bhlhb2', 'suffix': 'TCAAGG20NGAA_CACGTG_14_4'} 

current feature is: ('1mer',)...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4_1mer.tsv... 

[{'fileid': 'bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5573362145804319, 'mse_test': 0.005370543345655478, 'mae_test': 0.05703098149294358, 'best_alpha': 11.62553860384434}, {'fileid': 'bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5750518610170807, 'mse_test': 0.0053880287107389594, 'mae_test': 0.05814932643324112, 'best_alpha': 19.663405336538297}, {'fileid': 'bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5981395334296258, 'mse_test': 0.004711157283238865, 'mae_test': 0.05401264508246313, 'best_alpha': 12.480133868662266}, {'fileid': 'bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4', 'pos': '

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_1mer.tsv... 

Will load the additional feature DNaseI
DNaseI data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_DNaseI_0.tsv... 

Will load the additional feature twistDisp
twistDisp data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_twistDisp_0.tsv... 

Will load the additional feature NPP
NPP data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_NPP_0.tsv... 

Will load the additional feature stiffness
stiffness data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_stiffness_0.tsv... 

Will load the additional feature trxDi
trxDi data loaded from /SynologyNAS/00

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_TCF4_TGCGAA20NGA_CACCTG_14_3_1mer.tsv... 

[{'fileid': 'bHLH_TCF4_TGCGAA20NGA_CACCTG_14_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4723384760432605, 'mse_test': 0.008398757707448279, 'mae_test': 0.07505664908963275, 'best_alpha': 25.12349273286128}, {'fileid': 'bHLH_TCF4_TGCGAA20NGA_CACCTG_14_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.49873104659930334, 'mse_test': 0.008049822750296781, 'mae_test': 0.07271947625836965, 'best_alpha': 55.353951606628044}, {'fileid': 'bHLH_TCF4_TGCGAA20NGA_CACCTG_14_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.48798166619213945, 'mse_test': 0.008360067236899681, 'mae_test': 0.07469350248509256, 'best_alpha': 33.15157001298784}, {'fileid': 'bHLH_TCF4_TGCGAA20NGA_CACCTG_14_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4812888358117652, 'mse_test': 0.00815322597883359, 'mae_test': 0.07268388101083594, 'best_alpha': 41

twistDisp data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_TFAP4_TCACGT20NGCA_CAGCTG_12_4_twistDisp_0.tsv... 

Will load the additional feature NPP
NPP data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_TFAP4_TCACGT20NGCA_CAGCTG_12_4_NPP_0.tsv... 

Will load the additional feature stiffness
stiffness data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_TFAP4_TCACGT20NGCA_CAGCTG_12_4_stiffness_0.tsv... 

Will load the additional feature trxDi
trxDi data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_TFAP4_TCACGT20NGCA_CAGCTG_12_4_trxDi_0.tsv... 

[{'fileid': 'bHLH_TFAP4_TCACGT20NGCA_CAGCTG_12_4', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.7931191379192772, 'mse_test': 0.0025389206144206243, 'mae_test': 0.039172713234619315, 'best_alpha': 2.58

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_TFEB_TAGTTT20NCG_CACGTG_14_4_1mer.tsv... 

[{'fileid': 'bHLH_TFEB_TAGTTT20NCG_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5175490963716354, 'mse_test': 0.0072028701771130145, 'mae_test': 0.06743728993349239, 'best_alpha': 19.47412223157744}, {'fileid': 'bHLH_TFEB_TAGTTT20NCG_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5436133966465146, 'mse_test': 0.006319134271416566, 'mae_test': 0.06436369383143822, 'best_alpha': 25.286025764528063}, {'fileid': 'bHLH_TFEB_TAGTTT20NCG_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4685849460566951, 'mse_test': 0.006695963327128137, 'mae_test': 0.06491805157125494, 'best_alpha': 17.735758638999982}, {'fileid': 'bHLH_TFEB_TAGTTT20NCG_CACGTG_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.5269548441250627, 'mse_test': 0.006393094744103476, 'mae_test': 0.06401589358446537, 'best_alpha': 2

...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/homeodomain_LBX2_TAATTC20NTTA_TAAT_12_4_1mer.tsv... 

[{'fileid': 'homeodomain_LBX2_TAATTC20NTTA_TAAT_12_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6047328454603793, 'mse_test': 0.0051572037627798375, 'mae_test': 0.0548363402027978, 'best_alpha': 20.43911524515519}, {'fileid': 'homeodomain_LBX2_TAATTC20NTTA_TAAT_12_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6213166590533183, 'mse_test': 0.004824669933375874, 'mae_test': 0.055173714313287776, 'best_alpha': 29.328743589600503}, {'fileid': 'homeodomain_LBX2_TAATTC20NTTA_TAAT_12_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6031823780155076, 'mse_test': 0.005037461493906721, 'mae_test': 0.05539052708680272, 'best_alpha': 19.854528221151202}, {'fileid': 'homeodomain_LBX2_TAATTC20NTTA_TAAT_12_4', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6218814308954221, 'mse_test': 0.004469973045259472, 'mae_test': 0.0515

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/homeodomain_LHX2_TCCAGT30NGAC_TAAT_10_3_1mer.tsv... 

Will load the additional feature DNaseI
DNaseI data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX2_TCCAGT30NGAC_TAAT_10_3_DNaseI_0.tsv... 

Will load the additional feature twistDisp
twistDisp data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX2_TCCAGT30NGAC_TAAT_10_3_twistDisp_0.tsv... 

Will load the additional feature NPP
NPP data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX2_TCCAGT30NGAC_TAAT_10_3_NPP_0.tsv... 

Will load the additional feature stiffness
stiffness data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX2_TCCAGT30NGAC_TAAT_10_3_stiffness_0.tsv... 

Will load the additional feature trxDi
trx

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/homeodomain_LHX6_TCGGGG20NGGT_TAAT_10_3_1mer.tsv... 

[{'fileid': 'homeodomain_LHX6_TCGGGG20NGGT_TAAT_10_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.43184153316888685, 'mse_test': 0.003720677854493053, 'mae_test': 0.047907427774320735, 'best_alpha': 6.300280304250382}, {'fileid': 'homeodomain_LHX6_TCGGGG20NGGT_TAAT_10_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.431193291967224, 'mse_test': 0.0038432052380295402, 'mae_test': 0.04798521802601009, 'best_alpha': 12.241018759935384}, {'fileid': 'homeodomain_LHX6_TCGGGG20NGGT_TAAT_10_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4239308329872623, 'mse_test': 0.004260716243322377, 'mae_test': 0.04978875496166184, 'best_alpha': 7.644985663682395}, {'fileid': 'homeodomain_LHX6_TCGGGG20NGGT_TAAT_10_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.43194795046642187, 'mse_test': 0.003960878617564714, 'mae_test': 0.048

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/homeodomain_LHX9_TATTGT20NGTG_TAAT_12_3_1mer.tsv... 

Will load the additional feature DNaseI
DNaseI data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX9_TATTGT20NGTG_TAAT_12_3_DNaseI_0.tsv... 

Will load the additional feature twistDisp
twistDisp data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX9_TATTGT20NGTG_TAAT_12_3_twistDisp_0.tsv... 

Will load the additional feature NPP
NPP data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX9_TATTGT20NGTG_TAAT_12_3_NPP_0.tsv... 

Will load the additional feature stiffness
stiffness data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/homeodomain_LHX9_TATTGT20NGTG_TAAT_12_3_stiffness_0.tsv... 

Will load the additional feature trxDi
trx

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/homeodomain_MEOX1_TACCTA20NCG_TAAT_10_3_1mer.tsv... 



In [27]:
dd = pd.DataFrame([d for sl in results_df_f_combined for d in sl])

dd.to_csv("/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/MLR_L2_features_merged_interaction.tsv",
         sep="\t", index=False)

In [19]:
pd.DataFrame(dd)

,fileid,pos,features,r2_test,mse_test,mae_test,best_alpha
0,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.773803,0.003951,0.047663,10.0
1,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.773803,0.003951,0.047663,10.0
2,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.773803,0.003951,0.047663,10.0
3,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.773803,0.003951,0.047663,10.0
4,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.773803,0.003951,0.047663,10.0
...,...,...,...,...,...,...,...
4295,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.748790,0.003838,0.048977,2.0
4296,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.748790,0.003838,0.048977,2.0
4297,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.748790,0.003838,0.048977,2.0
4298,TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4,all,"(1mer, DNaseI, twistDisp, NPP, stiffness, trxDi)",0.748790,0.003838,0.048977,2.0


array([-100.        ,  -98.99497487,  -97.98994975,  -96.98492462,
        -95.9798995 ,  -94.97487437,  -93.96984925,  -92.96482412,
        -91.95979899,  -90.95477387,  -89.94974874,  -88.94472362,
        -87.93969849,  -86.93467337,  -85.92964824,  -84.92462312,
        -83.91959799,  -82.91457286,  -81.90954774,  -80.90452261,
        -79.89949749,  -78.89447236,  -77.88944724,  -76.88442211,
        -75.87939698,  -74.87437186,  -73.86934673,  -72.86432161,
        -71.85929648,  -70.85427136,  -69.84924623,  -68.84422111,
        -67.83919598,  -66.83417085,  -65.82914573,  -64.8241206 ,
        -63.81909548,  -62.81407035,  -61.80904523,  -60.8040201 ,
        -59.79899497,  -58.79396985,  -57.78894472,  -56.7839196 ,
        -55.77889447,  -54.77386935,  -53.76884422,  -52.7638191 ,
        -51.75879397,  -50.75376884,  -49.74874372,  -48.74371859,
        -47.73869347,  -46.73366834,  -45.72864322,  -44.72361809,
        -43.71859296,  -42.71356784,  -41.70854271,  -40.70351

In [ ]:
current feature is: ('1mer',)...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_1mer.tsv... 

[{'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.7036270840406365, 'mse_test': 0.006529026361283513, 'mae_test': 0.06119820076172802, 'best_alpha': 66.73672230356594}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6470494574834404, 'mse_test': 0.006454948507883834, 'mae_test': 0.06474292093748152, 'best_alpha': 34.348452963371095}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6476805803870389, 'mse_test': 0.005664352521794867, 'mae_test': 0.06009161425661814, 'best_alpha': 51.89709263556566}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.659616356345684, 'mse_test': 0.004997101961932499, 'mae_test': 0.056628179479259175, 'best_alpha': 43.1844174494043}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.7074827488447825, 'mse_test': 0.005428631245377078, 'mae_test': 0.05928866942753071, 'best_alpha': 47.11233472668556}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6732873164129476, 'mse_test': 0.005796666459683962, 'mae_test': 0.060279498981966044, 'best_alpha': 37.95909618044989},
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6721230115477281, 'mse_test': 0.005670420314729436, 'mae_test': 0.05949953669955408, 'best_alpha': 35.81878269030634}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6594335054908622, 'mse_test': 0.00721850174891553, 'mae_test': 0.06755460637519879, 'best_alpha': 31.790727035544535}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.710715442105734, 'mse_test': 0.005498312528554931, 'mae_test': 0.059002118653747174, 'best_alpha': 30.682968872179586}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6556995427488366, 'mse_test': 0.005535959224987777, 'mae_test': 0.05906716017267135, 'best_alpha': 30.09509365263908}]


current feature is: ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi')...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_1mer.tsv... 

Will load the additional feature DNaseI
DNaseI data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_DNaseI_0.tsv... 

Will load the additional feature twistDisp
twistDisp data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_twistDisp_0.tsv... 

Will load the additional feature NPP
NPP data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_NPP_0.tsv... 

Will load the additional feature stiffness
stiffness data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_stiffness_0.tsv... 

Will load the additional feature trxDi
trxDi data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_FLEX/bHLH_MAX_TGACCT20NGA_CACGTG_12_3_trxDi_0.tsv... 

[{'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.83556700608015, 'mse_test': 0.003622420586214008, 'mae_test': 0.04555572452144348, 'best_alpha': 16.362240376216743}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8167931949411108, 'mse_test': 0.0033505841484676485, 'mae_test': 0.04521363223359314, 'best_alpha': 13.615287156371553}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8003033538268254, 'mse_test': 0.00321058715011401, 'mae_test': 0.04319408244452127, 'best_alpha': 18.316925141081658},
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8044534636480642, 'mse_test': 0.002870778307566815, 'mae_test': 0.042738135871096664, 'best_alpha': 10.185951414410619}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8203891193245392, 'mse_test': 0.003333277729753851, 'mae_test': 0.04161595771462703, 'best_alpha': 3.1296739845529484},
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8293149876838259, 'mse_test': 0.0030283614189722274, 'mae_test': 0.04161260472612433, 'best_alpha': 14.33610873288389}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8398649412092649, 'mse_test': 0.002769432203076235, 'mae_test': 0.0413606826621068, 'best_alpha': 5.945040683613211}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.7658407794156696, 'mse_test': 0.004963138683824148, 'mae_test': 0.045321358536732956, 'best_alpha': 0.1}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8468458413221733, 'mse_test': 0.0029109380590109798, 'mae_test': 0.042553724675056075, 'best_alpha': 15.389958456025846}, 
 {'fileid': 'bHLH_MAX_TGACCT20NGA_CACGTG_12_3', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8276211342564375, 'mse_test': 0.0027716558369536696, 'mae_test': 0.037892273463929704, 'best_alpha': 8.313501065662443}]

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def encode_data_all_features_pp(fileprefix, data_dir="/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS", 
                features=("1mer",), w=0):
    try:
        # Load 1mer data
        ohe_file = os.path.join(data_dir, "ROHS_MWORDS_1mer", f"{fileprefix}_1mer.tsv")
        ohe_df = pd.read_csv(ohe_file, header=None, sep="\t")
        ohe_data = {row[0]: {'aff': float(row[0].split('__')[1]), 'feat': row.iloc[1:].tolist()} for _, row in ohe_df.iterrows()}
        print(f"1mer data loaded from {ohe_file}... \n")
    except FileNotFoundError as fe:
        raise FileNotFoundError(f"File {ohe_file} not found: {fe}")

    combined_data = ohe_data.copy()

    # Load and process additional features if specified
    if len(features) > 1:
        all_scaled_feats = {ids: [] for ids in ohe_data.keys()}

        for feature in features[1:]:
            print(f"Will load the additional feature {feature}")
            try:
                flex_feat_file = os.path.join(data_dir, "ROHS_MWORDS_FLEX", f"{fileprefix}_{feature}_{w}.tsv")
                flex_df = pd.read_csv(flex_feat_file, sep="\t", header=None)

                for ids in ohe_data.keys():
                    flex_df_row = flex_df[flex_df[0] == ids]
                    if not flex_df_row.empty:
                        flex_df_feat = np.array(flex_df_row.iloc[0, 1:].tolist())  # first column is id

                        # Calculate interaction terms for this feature using NumPy
#                         interaction_terms = flex_df_feat[:-1] * flex_df_feat[1:]

                        # Append both the original and interaction terms to the list
                        all_scaled_feats[ids].extend(flex_df_feat.tolist() + interaction_terms.tolist())
                print(f"{feature} data loaded from {flex_feat_file}... \n")
            except FileNotFoundError as fe:
                raise FileNotFoundError(f"File {flex_feat_file} not found: {fe}")
            except Exception as e:
                raise Exception(f"Error in extending feature {feature} from {flex_feat_file}: {e}")

        # Convert the dictionary to a DataFrame for scaling
        all_scaled_feats_df = pd.DataFrame.from_dict(all_scaled_feats, orient='index')

        # Scale the additional features using StandardScaler
        scaler = StandardScaler()
        scaled_feats = scaler.fit_transform(all_scaled_feats_df)

        # Merge scaled features with the 1mer features
        for i, ids in enumerate(ohe_data.keys()):
            combined_data[ids]['feat'].extend(scaled_feats[i].tolist())

    # Define X, y
    X = np.array([combined_data[x]['feat'] for x in combined_data.keys()])
    y = np.array([combined_data[x]['aff'] for x in combined_data.keys()])

    return X, y

# Example usage:
# X, y = encode_data("example_prefix", features=("1mer", "2mer", "3mer"))